#### StackOverflow 2019 Developer Survey Analysis  #### Full ETL + Analytics in One Notebook

This notebook performs:
✔ Extract → Transform → Load  
✔ Gender Normalization  
✔ Continent Mapping  
✔ Insights for all 6 assignment questions

---

#### QUESTIONS ANSWERED

1️. Average age when developers wrote first line of code  
2️. % of developers who know Python per country  
3️. Average salary per continent  
4️. Most desired language for 2020  
5️. Coding as hobby based on gender + continent  
6️. Job & Career satisfaction based on gender + continent


In [1]:
import pandas as pd
import pycountry_convert as pc
from collections import Counter

df = pd.read_csv("data/raw/survey_results_public.csv")

print("TOTAL ROWS:", len(df))
df.head(4)

TOTAL ROWS: 88883


,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy


#### STEP 1: Normalize Gender (MAN / WOMAN / OTHERS)


In [ ]:
d

In [2]:
def normalize_gender(value):
    if pd.isna(value):
        return "OTHERS"
    v = value.strip().lower()
    if v in ["man", "male"]:
        return "MAN"
    if v in ["woman", "female"]:
        return "WOMAN"
    return "OTHERS"

df["GenderNorm"] = df["Gender"].apply(normalize_gender)

df["GenderNorm"].value_counts()


GenderNorm
MAN       77919
WOMAN      6344
OTHERS     4620
Name: count, dtype: int64

#### STEP 2: Age when first code was written


In [3]:
df["Age1stCode"] = pd.to_numeric(df["Age1stCode"], errors="coerce")
avg_age = df["Age1stCode"].mean()
avg_age


np.float64(15.456446353114092)

#### STEP 3: Python Knowledge Percentage by Country


In [ ]:
df["KnowsPython"] = df["LanguageWorkedWith"].str.contains("Python", na=False)

python_pct = (
    df.groupby("Country")["KnowsPython"].mean() * 100
).reset_index().rename(columns={"KnowsPython":"PythonPercentage"})

python_pct.head(10)


,Country,PythonPercentage
0,Afghanistan,18.181818
1,Albania,26.744186
2,Algeria,29.850746
3,Andorra,0.000000
4,Angola,40.000000
5,Antigua and Barbuda,0.000000
6,Argentina,34.538879
7,Armenia,32.558140
8,Australia,41.513400
9,Austria,40.286055


#### STEP 4: Add Continent Column


In [5]:
def get_continent(country):
    try:
        code = pc.country_name_to_country_alpha2(country)
        cont = pc.country_alpha2_to_continent_code(code)
        return cont
    except:
        return None

df["Continent"] = df["Country"].apply(get_continent)
df["Continent"].value_counts()


Continent
EU    35789
NA    25494
AS    18345
SA     3434
AF     2732
OC     2434
Name: count, dtype: int64

#### STEP 5: Average Salary per Continent


In [6]:
df["ConvertedComp"] = pd.to_numeric(df["ConvertedComp"], errors="coerce")

salary_continent = (
    df.groupby("Continent")["ConvertedComp"].mean().reset_index()
)

salary_continent


,Continent,ConvertedComp
0,AF,34925.546400
1,AS,36476.492550
2,EU,97116.182129
3,NA,225667.247418
4,OC,166950.955791
5,SA,33010.825072


#### STEP 6: Most Desired Language for NEXT YEAR (2020 prediction)


In [7]:
langs = df["LanguageDesireNextYear"].dropna().str.split(";")

flat = [item for sub in langs for item in sub]

Counter(flat).most_common(1)[0]


('JavaScript', 44739)

#### STEP 7: Hobby Coding by Gender + Continent


In [8]:
hobby_breakdown = (
    df.groupby(["GenderNorm","Continent"])["Hobbyist"]
    .value_counts(normalize=True)
    .rename("Percentage")
    .reset_index()
)

hobby_breakdown.head()


,GenderNorm,Continent,Hobbyist,Percentage
0,MAN,AF,Yes,0.796512
1,MAN,AF,No,0.203488
2,MAN,AS,Yes,0.790470
3,MAN,AS,No,0.209530
4,MAN,EU,Yes,0.831115


#### STEP 8: Job & Career Satisfaction by Gender + Continent

In [9]:
satisfaction_map = {
    "Very dissatisfied": 1,
    "Slightly dissatisfied": 2,
    "Neither satisfied nor dissatisfied": 3,
    "Slightly satisfied": 4,
    "Very satisfied": 5
}

df["JobSat"] = df["JobSat"].map(satisfaction_map)
df["CareerSat"] = df["CareerSat"].map(satisfaction_map)

job_career = df.groupby(["GenderNorm","Continent"])[["JobSat","CareerSat"]].mean().reset_index()

job_career


,GenderNorm,Continent,JobSat,CareerSat
0,MAN,AF,3.465483,3.776173
1,MAN,AS,3.428139,3.681049
2,MAN,EU,3.701014,3.931167
3,MAN,NA,3.868518,4.174292
4,MAN,OC,3.786732,4.037695
5,MAN,SA,3.467821,3.719093
6,OTHERS,AF,3.213115,3.232877
7,OTHERS,AS,3.400498,3.594218
8,OTHERS,EU,3.719101,3.814383
9,OTHERS,NA,3.669903,3.912058


In [13]:
import plotly.express as px
df["KnowsPython"] = df["LanguageWorkedWith"].str.contains("Python", na=False)

python_pct_country = (
    df.groupby("Country")["KnowsPython"].mean() * 100
).reset_index().rename(columns={"KnowsPython": "PythonPercentage"})


top15 = python_pct_country.sort_values("PythonPercentage", ascending=False).head(15)

fig = px.bar(
    top15,
    x="Country",
    y="PythonPercentage",
    title="Top 15 Countries Who Know Python",
    text="PythonPercentage"
)

fig.update_traces(texttemplate="%{text:.1f}%", textposition="outside")
fig.update_layout(yaxis_title="Python %", xaxis_title="Country")

fig.show()


In [11]:
import nbformat
nbformat.__version__


'5.10.4'

In [14]:
df.to_csv("data/processed/cleaned_survey.csv", index=False)
